In [ ]:
# 

# Create target object and call it y

# Create X

# Split into validation and training data

# Specify Model

# Fit Model

# Make validation predictions and calculate mean absolute error

# Show a prediction result and real data.

### Importing libraries and Reading the data.

In [2]:
# Importing the libraries
import pandas as pd
import numpy as np

In [3]:
# Impoirting the dataset
house_prices_dataset_path = "./data/house-prices-advanced-regression-techniques/sample_submission.csv"
house_prices_test_path = "./data/house-prices-advanced-regression-techniques/test.csv"
house_prices_train_path = "./data/house-prices-advanced-regression-techniques/train.csv"
house_data_discription_path = "./data/house-prices-advanced-regression-techniques/data_description.txt"

house_prices = pd.read_csv(house_prices_dataset_path)
house_prices_test = pd.read_csv(house_prices_test_path)
house_prices_train = pd.read_csv(house_prices_test_path)

In [4]:
# Exporting the data (name of columns) to .xlsx file to more convenient work.
df = pd.DataFrame(house_prices_train.columns)
df.to_excel('./auxiliary_files/show_columns.xlsx', sheet_name='Column Names')

### Classes

In [5]:
class Nan_counter:
    """Counting NaN(true) and NaN(false) values"""  
    def __init__(self, data_set):
        self.data = data_set

    def is_nan(self, col_name):
        """returning number of the NaN cells. In view True and False:
        True     1352
        False     107
        Name: column_name, dtype: int64"""
        return pd.isnull(self.data[self.col_name]).value_counts()
    
    def missing_values(self, accuracy):
        """accuracy in percentage
        #.missing_values(.01) - .01 is an accuracy in percent. 
        IOW, takes all values that less then .01 percent."""
        i = self.data.isnull().sum()
        j = len(self.data)
        y = round(i/j, 2)  
        return dict(y[y <= accuracy])

In [6]:
class Nan_columns:
    """
    - Finding the column(s) in dataframe that stored need number of NaN values,
    - Create a list with those columns"""
    def __init__(self, data_set, min_required_nan):
        self.data = data_set
        self.min_required_nan = min_required_nan
        
    def min_nan_number_in_column(self):
        """finding the columns that stroed NaN where each column stored less or equal then # NaN values."""
        columns_of_dataset = self.data.columns.tolist()
        best_columns = []

        for i in columns_of_dataset:
            nan_sum = self.data[f"{i}"].isna().sum()            
            if nan_sum <= self.min_required_nan:
                best_columns.append(i)
        
        return best_columns

In [7]:
class Nan_string:
    """Show the string(s) that stored NaN value(s)"""
    def __init__(self, dataframe):
        self.df = dataframe
        
    def show_nan_string(self):
        """Return only those strings that contain NaN values"""
        # creating and filling a dictionary (#str: col_name)
        nan_str_col = dict([(x, self.df.columns[y]) for x, y in zip(*np.where(self.df.isna()))])
        
        # takes only keys from dictionary
        keys = list(nan_str_col.keys())

        return self.df.loc[keys]
        
    def show_nan_dict(self):
        """Return only number of the string and a name of the column that string(s) storing NaN values"""
        return dict([(x, self.df.columns[y]) for x, y in zip(*np.where(self.df.isna()))])

In [8]:
class Missing_column:
    """Differece between two lists(dataset and custom_sample."""
    def find_miss_col(self, col_dataframe, col_custom_sample):
        diff = set(col_dataframe.columns) - set(col_custom_sample)
        return diff

### Review Data

In [9]:
# Counting the unique values inside column.
print(house_prices_test['LandContour'].value_counts(sort=False))

Low      24
Lvl    1311
HLS      70
Bnk      54
Name: LandContour, dtype: int64


In [10]:
# reviewing all columns and find number NaN strings inside.
#.missing_values(.01) - .01 is an accuracy in percent. IOW, takes all values that less then .01 percent.
x = Nan_counter(house_prices_test).missing_values(.001)

features_01 = list(x)

print(len(features_01))
print(features_01)

62
['Id', 'MSSubClass', 'MSZoning', 'LotArea', 'Street', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'GarageCars', 'GarageArea', 'PavedDrive', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold', 'SaleType', 'SaleCondition']


In [11]:
# Select dataset columns which storing less or equal than # NaN strings.
features_02 = Nan_columns(house_prices_test, 0).min_nan_number_in_column()

print(len(features_02))
print(features_02)

47
['Id', 'MSSubClass', 'LotArea', 'Street', 'LotShape', 'LandContour', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle', 'RoofMatl', 'ExterQual', 'ExterCond', 'Foundation', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'PavedDrive', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold', 'SaleCondition']


In [12]:
# finding the missing values columns from dataset to restructure them to improve the forcast accuracy.
i = Missing_column().find_miss_col(house_prices_test, features_02)
missing_columns = list(i)

print(house_prices_test[missing_columns].isnull().sum())

BsmtFinType1      42
LotFrontage      227
GarageFinish      78
GarageArea         1
MiscFeature     1408
TotalBsmtSF        1
GarageCars         1
GarageQual        78
BsmtExposure      44
BsmtUnfSF          1
Exterior1st        1
BsmtHalfBath       2
KitchenQual        1
PoolQC          1456
FireplaceQu      730
Utilities          2
BsmtFullBath       2
BsmtFinSF1         1
BsmtCond          45
GarageType        76
Functional         2
MasVnrArea        15
Fence           1169
Alley           1352
Exterior2nd        1
GarageYrBlt       78
BsmtFinType2      42
GarageCond        78
BsmtFinSF2         1
SaleType           1
BsmtQual          44
MasVnrType        16
MSZoning           4
dtype: int64


In [13]:
# Show str_num and col_name that storing NaN values.
show_nan_string = Nan_string(house_prices_test[missing_columns])
# show_nan_string.show_nan_dict()

In [30]:
# Observing those strings in DataFrame that storing NaN values. String evaluation, visually analysis.
show_nan_string.show_nan_string([:5]

,BsmtFinType1,LotFrontage,GarageFinish,GarageArea,MiscFeature,TotalBsmtSF,GarageCars,GarageQual,BsmtExposure,BsmtUnfSF,...,Alley,Exterior2nd,GarageYrBlt,BsmtFinType2,GarageCond,BsmtFinSF2,SaleType,BsmtQual,MasVnrType,MSZoning
0,False,False,False,False,True,False,False,False,False,False,...,True,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,True,False,False,False,False,False,False,False,False,False
2,False,False,False,False,True,False,False,False,False,False,...,True,False,False,False,False,False,False,False,False,False
3,False,False,False,False,True,False,False,False,False,False,...,True,False,False,False,False,False,False,False,False,False
4,False,False,False,False,True,False,False,False,False,False,...,True,False,False,False,False,False,False,False,False,False


### Cleaning the data